# Generating a Spectrum-Through-a-Cube Plot
This is a tutorial for a simple spectrum-through-a-cube plot.

## Authors
Jeff Mangum, Kris Stern

## Learning Goals
1. To produce a publication-grade spectrum with emission lines labeled with transition markers.
2. To enter inputs using a YAML file.
3. To read positions and areal patterns with which to derive spectra from some ds9 region. 

## Keywords
data cubes, fits image, spectroscopy

## Companion Content
[Jeff's GitHub repo with all the source materials](https://github.com/jmangum/spectrumplot)

## Summary
Basically, we integrate a datacube across some pre-defined region and to generate a spectrum plot that way. A neat trick using a YAML file is introduced, explained side-by-side by definitions of variables pythonic to illustrate what has been achieved using the YAML file. 

## System requirements
You will need to install some additional packages, including: [lineid_plot](https://pypi.org/project/lineid_plot/), [PyYAML](https://pypi.org/project/PyYAML/), and the [regions](https://astropy-regions.readthedocs.io/en/latest/installation.html) package.

## Caveats on some software used
It should be noted that some of the software use is not part of the Astropy ecosystem. But since Astropy is still developing some functionalities are yet to be made available, hence the choice of software in this context. 

First of all, open this file in the directory containing all the files from the repo in the ["Companion Content"](#companion-content). Next, you will need to import some required packages, as shown below:

Import the packages to set up the coding environment...

In [ ]:
!python -m pip install git+https://github.com/pyspeckit/pyspeckit.git#egg=pyspeckit
!python -m pip install regions

In [ ]:
# Import all necessary packages
import matplotlib 
from matplotlib import pyplot as plt
%matplotlib inline

import astropy
import astropy.units as u 
import numpy as np 
import regions 
from astropy.utils import data 
from spectral_cube import SpectralCube 
import pyspeckit 
from astropy.io.misc import yaml 
from astropy.utils.data import download_file

After that, define the input .yaml file which contains all the input variables needed to run the code properly. YAML files are data-oriented, and is short for "YAML Ain't Markup Language" (recursive). It is commonly used for configuration files, but can also be used in many other applications where data is being stored or transmitted. In our context, the YAML file concerned is used for storing data in an organized manner. 

In [ ]:
# "infile" is the input yaml file.  
infile = './CubeSpectrumPlotExampleInput.yaml'

Once the infile is ready, load it with the load method of `yaml` as below. Change the directory with the `os` command lines by entering the working directory if needed: 

In [ ]:
with open(infile) as fh:
    params = yaml.load(fh)

After opening the yaml file and loading the info contained, you are ready to define the following variables. Note that this example uses an input list of transitions to mark which transitions outside the frequency range of the FITS file used (to be able to use the same YAML file with other FITS cubes from this target) to include. You can ignore any warnings about "skipped out-of-bounds lines".

In [ ]:
cubefile = params['cubefile'] # Cube from which the spectrum is to be extracted 
regfile = params['regfile'] # Regions file containing positions through which spectra can be extracted 
target = params['target'] # Target name for plot annotation 
figfile = params['figfile'] # Output figure file name 
velconvention = params['velconvention'] # Velocity axis convention used for spectrum 
regplot = int(params['regplot']) # Which region number to produce spectrum towards 
smoothfact = int(params['smoothfact']) # Smooth spectrum to smoothfact resolution in km/s 
yminval = float(params['yminval']) # Intensity axis minimum value 
ymaxval = float(params['ymaxval']) # Intensity axis maximum value

Alternatively, to spell out the details, we can define the above parameters needed pythonically and individually as follows: 

In [ ]:
cubefile = 'CubeSpectrumPlotExample.fits'  # This is the data cube we will be integrating over
regfile = 'LeroyNGC253Positions.reg'  # This is the file from which the positions are the spectra extracted
target = 'NGC253'  # The name of the scientific target for which spectrum belongs
figfile = 'CubeSpectrumPlotExample.png'  # The name of the output file
velconvention = 'optical'  # Velocity axis convention used for the spectrum, either 'radio' or 'optical'
regplot = int(5)  # Region number to produce spectrum towards, is an integer
smoothfact = int(10)  # The smoothfact resolution in km/s to be used to smooth the spectrum
yminval = float(-5)  # The mininum y-value, a float
ymaxval = float(145)  # the maximum y-value, a float

Using the second way is more pythonic in the definition of various variables. However, when considering also that ... 

Note that the input parameters in YAML file include:

* cubefile: FITS cube file name
* regfile: Regions file name
* figfile: File name for output figure file
* target: Target name used to annotate upper-left corner of plot
* velconvention: Velocity convention for spectral axis of plot (optical or radio)
* regplot: Region number in regions file at which to plot spectum
* smoothfact: Spectral smoothing factor
* yminval: Intensity axis minimum value
* ymaxval: Intensity axis maximum value
* linenames: Name tags for transitions marked in spectrum
* linexvals: Frequency values (in MHz) for linenames marked on spectrum
* linenames_sizes: Character sizes for linename annotations
* arrow_tips: Arrow tip sizes
* vregion: Nominal velcity for region plotted (to allow for centering of linexvals on linenames plotted)

Then we proceed to defining the line names, x values, sizes of the line names, and size of the arrow tips. These will be useful for the formatting of the spectrum plot to be generated, both for its readability and the aesthetics. 

In [ ]:
if params['linenames'] != None: 
    linenames = list(params['linenames'].split(", ")) # Line names to use for line ID markers
    linexvals = u.Quantity(list(map(float, params['linexvals'].split(", "))), u.MHz) # X-axis values for linenames
    linenames_sizes = np.array(list(map(int,params['linenames_sizes'].split(", ")))) # Size in points for each linename
    arrow_tips = list(map(int,params['arrow_tips'].split(", "))) # Size of arrow tips used in annotate
    # Define spectral line annotation box locations
    box_locs = [x+25 for x in arrow_tips]
vregion = u.Quantity(float(params['vregion']), u.km/u.s)

Next, we invoke SpectralCube to read the given cube FITS file. The sample cube has 3 dimensions: 2 spatial axes (in degrees) and one velocity (in m/s). 

In [ ]:
tutorialpath = 'http://data.astropy.org/tutorials/spectrumplot/'
datafilename1 = download_file(tutorialpath + cubefile, cache=True, show_progress=True)
cube = SpectralCube.read(datafilename1) # Read cube
print(cube)

From the data in the file "LeroyNGC253Positions.reg" we note the following line, and use it to construct a ds9 region to make a subcube. 

In [ ]:
cubespec = cube.subcube_from_ds9region('fk5; circle(11.88888,-25.28771,0.5")').mean(axis=(1, 2))

We can now convert the unit of the `cubespec` object made with `subcube_from_ds9region`, and store the HDU in a new variable `cubespec_hdu`. 

In [ ]:
import astropy.io.fits as fits
cubespec_mJyperbeam = cubespec*1000*u.mJy/u.Jy
cubespec_hdu = cubespec_mJyperbeam.hdu

Then, we can obtain the offset from the rest frequency for each emission line. The offset is dependent on the particular velocity defintion used. 

In [ ]:
restf = cubespec.wcs.wcs.restfrq*u.Hz 
if velconvention == 'optical': 
    offset = restf-(vregion).to(u.GHz,u.doppler_optical(restf)) 
elif velconvention == 'radio': 
    offset = restf-(vregion).to(u.GHz,u.doppler_radio(restf)) 
print(offset)

The following will define the spectral axis units using the specified velocity convention:

In [ ]:
spectral_axis = cubespec_mJyperbeam.with_spectral_unit(u.GHz, velocity_convention=velconvention, rest_value=restf).spectral_axis + offset

Next, we extract the desired spectrum from `cubespec_mJyperbeam` as `sp`. 

In [ ]:
sp = pyspeckit.Spectrum(xarr=spectral_axis, data=cubespec_mJyperbeam.value, header={})

The following will convert `sp` to the specified velocity convention read from the input yaml file. It should be noted there are three velocity definitions by convention - optical, radio, and relativistic. But for the situation considered only the optical velocity and radio velocity ones are relevant. For more information see for example https://www.eaobservatory.org/jcmt/instrumentation/heterodyne/velocity-considerations/

In [ ]:
if velconvention == 'optical':
    sp.xarr.convert_to_unit('km/s', refX=restf, equivalencies=u.doppler_optical(restf)) # Do this first so one can smooth to smoothfact km/s spectral resolution
elif velconvention == 'radio':
    sp.xarr.convert_to_unit('km/s', refX=restf, equivalencies=u.doppler_radio(restf))

Then, we smooth to smoothfact km/s, which is smoothfact over the cdelt channels.

In [ ]:
sp.smooth(smoothfact/np.abs(sp.xarr.cdelt(approx=True).value))

Converting to a freqency axis:

In [ ]:
sp.xarr.convert_to_unit('GHz')

Finally, it is on to plotting with PySpecKit's `plotter` function... 

In [ ]:
sp.plotter(ymin=yminval,ymax=ymaxval)
sp.plotter.label(xlabel='Rest Frequency (GHz)',ylabel='Flux Density (mJy beam$^{-1}$)') # X- and Y-axis labeling
sp.plotter.axis.plot([sp.xarr.min().value,sp.xarr.max().value],[0,0],color='k',linestyle='-',zorder=-5) # Draw a line at y=0
if params['linenames'] != None:
    sp.plotter.line_ids(linenames,linexvals,auto_yloc=False,auto_yloc_fraction=0.83,label1_size=9,arrow_tip=arrow_tips,box_loc=box_locs) # Plot line IDs read from yaml file
# NOTE: annotate must be called *after* line_ids
sp.plotter.axis.annotate(s=target,xy=(0.05,0.9),xycoords='axes fraction') # Annotate with source name
sp.plotter.savefig('spectrum_try.png') # Save to output file